# Napari image viewer

Uses the octopuslite dask image loader to automatically sort and load images from multichannel time lapse experiments. Can also use btrack to load associated tracks and segmentation images.

In [1]:
import napari
from octopuslite import DaskOctopusLiteLoader
import glob
import numpy as np

import numpy as np
from tqdm.auto import tqdm

def compute(dask_image_stack, N = None):
    
    """lazy hack to get around dask not loading transformed stacks in one go"""
    
    if N == None:
        N = (0, len(dask_image_stack) )
        
    return np.stack([frame.compute().astype(np.uint8) for frame in tqdm(dask_image_stack[N[0]:N[1]], total = N[1]-N[0])], axis = 0)

# Load images

In [23]:
# define experiment ID and select a position
expt = 'ND0017'
pos = 'Pos4'
# point to where the data is
root_dir = '/home/nathan/data/kraken/ras'
image_path = f'{root_dir}/{expt}/{pos}/{pos}_images'
transform_paths = glob.glob(f'{root_dir}/{expt}/{pos}/*.npy')
print(transform_paths)
tracks_path = f'{root_dir}/{expt}/{pos}/tracks.h5'

['/home/nathan/data/kraken/ras/ND0017/Pos4/transform_tensor.npy']


In [24]:
transform_path = transform_paths[0] #'/home/nathan/data/kraken/ras/ND0019/Pos2/mask_reversed_clipped_transform_tensor.npy'

In [25]:
# lazily load images
images = DaskOctopusLiteLoader(image_path,
                              transforms = transform_path,
                              crop = (1200,1600),
                              remove_background = True)
                              
images.channels

Using cropping: (1200, 1600)


[<Channels.BRIGHTFIELD: 0>,
 <Channels.GFP: 1>,
 <Channels.RFP: 2>,
 <Channels.IRFP: 3>,
 <Channels.MASK: 99>]

In [26]:
gfp = images['gfp']
rfp = images['rfp']#.compute().astype(np.uint8)
# irfp = images['irfp']
masks = images['mask']#.compute().astype(np.uint8)
#masks_old = images['mask_gfp']

### Add tracks

In [27]:
import btrack
from btrack.utils import tracks_to_napari

#### GFP tracks

In [28]:
with btrack.dataio.HDF5FileHandler(tracks_path, 
                                   'r', obj_type='obj_type_1'
                                  ) as h:
    gfp_tracks = h.tracks

[INFO][2022/04/11 04:12:15 pm] Opening HDF file: /home/nathan/data/kraken/ras/ND0017/Pos4/tracks.h5...


OSError: Unable to open file (unable to open file: name = '/home/nathan/data/kraken/ras/ND0017/Pos4/tracks.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

In [ ]:
gfp_tracks = [track for track in gfp_tracks if len(track)>20]

In [ ]:
gfp_tracks, gfp_properties, gfp_graph = tracks_to_napari(gfp_tracks, ndim = 2)

#### RFP tracks

In [ ]:
with btrack.dataio.HDF5FileHandler(tracks_path, 
                                   'r', obj_type='obj_type_2'
                                  ) as h:
    rfp_tracks = h.tracks

In [ ]:
rfp_tracks = [track for track in rfp_tracks if len(track)>20]

In [ ]:
rfp_tracks, rfp_properties, rfp_graph = tracks_to_napari(rfp_tracks, ndim = 2)

# Launch viewer

In [29]:
## number of frames to load
N = len(gfp)

In [30]:

viewer = napari.Viewer()
viewer.add_image(gfp[0:N].astype(np.uint8), name="gfp", 
                 contrast_limits=[0,255],
                 blending = 'additive', colormap = 'green')#, is_pyramid=False)
viewer.add_image(rfp[0:N].astype(np.uint8), name="rfp", 
                 contrast_limits=[0,55], 
                 blending = 'additive', colormap = 'magenta')#, is_pyramid=False)
# viewer.add_image(irfp[0:N].astype(np.uint8), name="irfp", 
#                  contrast_limits=[0,255], 
#                  blending = 'additive', colormap = 'cyan')#
viewer.add_labels(masks[0:N].astype(np.uint8), name = 'masks',
               blending = 'additive',)
# viewer.add_labels(masks_2[0:N].astype(np.uint8), name = 'masks filt',
#                blending = 'additive',)
# # viewer.add_labels(images['mask'].astype(np.uint8), name = 'masks original',
#                blending = 'additive',)
viewer.add_tracks(gfp_tracks)
viewer.add_tracks(rfp_tracks)

__init__.py (165): Public access to Window.qt_viewer is deprecated and will be removed in
v0.5.0. It is considered an "implementation detail" of the napari
application, not part of the napari viewer model. If your use case
requires access to qt_viewer, please open an issue to discuss.


<Tracks layer 'rfp_tracks' at 0x7fad8512d100>

In [15]:
stop

NameError: name 'stop' is not defined

# Bring images into memory?

In [ ]:
gfp

In [ ]:
gfp = compute(images['gfp'], N = (1200,1400))
rfp = compute(images['rfp'], N = (1200,1400))#.compute().astype(np.uint8)
# irfp = images['irfp']
masks = compute(images['mask'], N = (1200,1400))#.compute().astype(np.uint8)
#masks_2 = compute(images['mask_rfp'], N = 200)

# temp assigning all h5 files out to archive folder

In [17]:
files = glob.glob('/home/nathan/data/kraken/ras/ND****/Pos*/*.h5')

In [22]:
for fn in files:
    new_fn = (fn.split('ras/')[-1]).replace('/','_')
    new_path = '/home/nathan/data/kraken'
    break

['/home/nathan/data/kraken/', 'ND0013/Pos7/old_objects_type_2.h5']
